In [1]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


import warnings
import numpy as np
from collections import OrderedDict

import os

from lob_data_utils import lob, db_result, gdf_pca, model
from lob_data_utils.svm_calculation import lob_svm


sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
data_length = 24000
stock = '9761'
gdf_filename_pattern = 'gdf_{}_r{}_s{}_K50'
gdf_parameters = [(0.1, 0.1), (0.01, 0.1), (0.1, 0.5), (0.01, 0.5)]

In [4]:
df_log = pd.read_csv('../gdf_pca/res_log_que.csv')   
df_log = df_log[df_log['stock'] == int(stock)]
columns = [c for c in df_log.columns if 'matthews' in c or 'roc_auc' in c]
df_log[columns]

,matthews,roc_auc,test_matthews,test_roc_auc,train_matthews,train_roc_auc,train_val_matthews,train_val_roc_auc
35,0.146427,0.579966,0.107514,0.558467,0.125276,0.568539,0.125167,0.568139


In [6]:
gdf_dfs = []
for r, s in gdf_parameters:
    gdf_dfs.append(gdf_pca.SvmGdfResults(
        stock,  r=r, s=s, data_length=data_length, data_dir='~/shit2',
        gdf_filename_pattern=gdf_filename_pattern))

In [7]:
gdf_dfs[0].df.columns

Index(['Unnamed: 0', 'datetime', 'gdf_0', 'gdf_1', 'gdf_10', 'gdf_11',
       'gdf_12', 'gdf_13', 'gdf_14', 'gdf_15', 'gdf_16', 'gdf_17', 'gdf_18',
       'gdf_19', 'gdf_2', 'gdf_20', 'gdf_21', 'gdf_22', 'gdf_23', 'gdf_24',
       'gdf_25', 'gdf_26', 'gdf_27', 'gdf_28', 'gdf_29', 'gdf_3', 'gdf_30',
       'gdf_31', 'gdf_32', 'gdf_33', 'gdf_34', 'gdf_35', 'gdf_36', 'gdf_37',
       'gdf_38', 'gdf_39', 'gdf_4', 'gdf_40', 'gdf_41', 'gdf_42', 'gdf_43',
       'gdf_44', 'gdf_45', 'gdf_46', 'gdf_47', 'gdf_48', 'gdf_49', 'gdf_5',
       'gdf_6', 'gdf_7', 'gdf_8', 'gdf_9', 'mid_price', 'mid_price_indicator',
       'queue_imbalance', 'bid_price', 'ask_price', 'prev_queue_imbalance'],
      dtype='object')

In [8]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper
import tensorflow as tf
auc_roc = as_keras_metric(tf.metrics.auc)

In [ ]:
df = gdf_dfs[0].df
df_test = gdf_dfs[0].df_test

X_train = df[[gdf for gdf in gdf_dfs[0].df.columns if 'gdf' in gdf or 'queue' in gdf]].values
y_train = gdf_dfs[0].df['mid_price_indicator'].values
pca = PCA(n_components=3)
pca.fit(X_train)
X_train = pca.transform(X_train)

X_test = df_test[[gdf for gdf in gdf_dfs[0].df_test.columns if 'gdf' in gdf or 'queue' in gdf]].values
y_test = df_test['mid_price_indicator'].values
X_test = pca.transform(X_test)

#X_test= np.reshape(X_test, (X_test.shape[0], 1, 3))
#X_train = np.reshape(X_train, (X_train.shape[0], 1, 3))
print(X_train.shape)
print(X_test.shape)


In [ ]:
from sklearn.utils import class_weight
a = np.unique(y_train)
print(a)
class_weight.compute_class_weight('balanced', a, y_train)

In [ ]:
max_features = 1024
model = Sequential()
# model.add(LSTM(6))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc_roc])

model.fit(X_train, y_train, shuffle=False, 
          epochs=4, batch_size=50, class_weight={0: 0.71002219, 1: 1.69035041})
print(model.summary())

In [ ]:
pred_test = model.predict_classes(X_test)
pred = model.predict_classes(X_train)

print(metrics.roc_auc_score(y_test, pred_test), metrics.roc_auc_score(y_train, pred))
print(metrics.matthews_corrcoef(y_test, pred_test), metrics.matthews_corrcoef(y_train, pred))

In [14]:
model = Sequential()
model.add(LSTM(6))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc_roc])

In [15]:
epochs = 3
batch_size = 50
score = gdf_dfs[0].train_lstm(
    model, feature_name='pca_n_gdf_que_prev', plot_name='here.png',
    fit_kwargs={'epochs': epochs, 'batch_size': batch_size, 'verbose': 1, 
                'class_weight': {0: 0.71002219, 1: 1.69035041}, 'shuffle': False}, 
    compile_kwargs= { 'loss': 'binary_crossentropy', 'optimizer': 'adam', 'metrics': [auc_roc]})

Epoch 1/3
8725/8725 [==============================] - 3s 289us/step - loss: 0.6868 - auc: 0.5762
Epoch 2/3
8725/8725 [==============================] - 0s 46us/step - loss: 0.6869 - auc: 0.5854
Epoch 3/3
8725/8725 [==============================] - 0s 48us/step - loss: 0.6864 - auc: 0.5871
Epoch 1/3
8725/8725 [==============================] - 3s 333us/step - loss: 0.6780 - auc: 0.5763
Epoch 2/3
8725/8725 [==============================] - 0s 48us/step - loss: 0.6770 - auc: 0.5900
Epoch 3/3
8725/8725 [==============================] - 0s 56us/step - loss: 0.6767 - auc: 0.5912
Epoch 1/3
8725/8725 [==============================] - 3s 342us/step - loss: 0.6702 - auc: 0.5765
Epoch 2/3
8725/8725 [==============================] - 0s 51us/step - loss: 0.6696 - auc: 0.5923
Epoch 3/3
8725/8725 [==============================] - 0s 50us/step - loss: 0.6693 - auc: 0.5935
Epoch 1/3
8725/8725 [==============================] - 3s 365us/step - loss: 0.6671 - auc: 0.5836
Epoch 2/3
8725/8725 [=====

In [16]:
score

{'precision': 0.3645214368099968,
 'f1': 0.4035936534246898,
 'recall': 0.49424392816898133,
 'roc_auc': 0.5724031829164193,
 'kappa': 0.1279464530598284,
 'matthews': 0.13745705998636343,
 'train_val_precision': 0.3616774880844805,
 'train_val_f1': 0.40701070023442754,
 'train_val_recall': 0.5022124514333182,
 'train_val_roc_auc': 0.5683271107492425,
 'train_val_kappa': 0.12196590678046657,
 'train_val_matthews': 0.13087735528486508,
 'train_precision': 0.3435976392096484,
 'train_f1': 0.46252158894645934,
 'train_recall': 0.7073428420496566,
 'train_roc_auc': 0.5698696458768994,
 'train_kappa': 0.1069175937058614,
 'train_matthews': 0.13069376092927196,
 'stock': '9761',
 'kernel': 'lstm',
 'features': 'pca_n_gdf_que_prev',
 'pca_components': 4,
 'test_precision': 0.3461939520333681,
 'test_f1': 0.4598337950138504,
 'test_recall': 0.6845360824742268,
 'test_roc_auc': 0.5610180412371134,
 'test_kappa': 0.0956365518211113,
 'test_matthews': 0.11447384390129056}

In [ ]:
## With validation

In [15]:
model = Sequential()
model.add(LSTM(12, input_shape=(10, 4)))
model.add(Dense(12, activation='tanh'))
model.add(Dense(6, activation='tanh'))
model.add(Dense(3, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc_roc])

In [16]:
epochs = 3
batch_size = 50
score = gdf_dfs[0].train_lstm(
    model, feature_name='pca_n_gdf_que_prev', plot_name='here.png', n_steps=10,
    fit_kwargs={'epochs': epochs, 'batch_size': batch_size, 'verbose': 1, 'shuffle': False}, 
    class_weight={0: 0.71002219, 1: 1.69035041},
    compile_kwargs= { 'loss': 'binary_crossentropy', 'optimizer': 'adam', 'metrics': [auc_roc]})

Epoch 1/3
8720/8720 [==============================] - 10s 1ms/step - loss: 0.6927 - auc: 0.4982
Epoch 2/3
8720/8720 [==============================] - 8s 916us/step - loss: 0.6873 - auc: 0.5620
Epoch 3/3
8720/8720 [==============================] - 8s 896us/step - loss: 0.6847 - auc: 0.5750
Epoch 1/3
8720/8720 [==============================] - 12s 1ms/step - loss: 0.6751 - auc: 0.5802
Epoch 2/3
8720/8720 [==============================] - 8s 878us/step - loss: 0.6739 - auc: 0.5970 0s - loss: 0.6722 - auc: 0.5
Epoch 3/3
8720/8720 [==============================] - 6s 669us/step - loss: 0.6734 - auc: 0.5988
Epoch 1/3
8720/8720 [==============================] - 8s 952us/step - loss: 0.6663 - auc: 0.5873 1s - loss: 0.66
Epoch 2/3
8720/8720 [==============================] - 6s 634us/step - loss: 0.6659 - auc: 0.6026
Epoch 3/3
8720/8720 [==============================] - 5s 597us/step - loss: 0.6657 - auc: 0.6034
Epoch 1/3
8720/8720 [==============================] - 8s 936us/step - loss

In [18]:
score['matthews'], score['test_matthews']

(0.14407964291768027, 0.11959443708797061)